## Wiktionary extractor

### Multilingual extractions

This pipeline takes an input of a txt file, that contains the data manually copied from a Wiktionary translations section. The tool will clean the data using regular expressions, and organize it in a neat csv, ready for plotting or further analysis.

In [103]:
# Import libraries
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import regex as re

path_in_wiktionary = "data/wiktionary/"

### Build dataset

In [104]:
key = 'silk'

In [105]:
# Create lists of files and senses
list_of_files = []
for file in os.listdir(path_in_wiktionary):
    if file.startswith(key) and file.endswith('.txt'):
        list_of_files.append(file)

print(list_of_files)

list_of_senses = []
for file in list_of_files:
    sense = re.sub('.*-','',file)
    sense = re.sub('\.txt','',sense)
    list_of_senses.append(sense)

print(list_of_senses)

['silk-fabric.txt', 'silk-fiber.txt']
['fabric', 'fiber']


In [106]:
df_key = pd.DataFrame()

for file, sense in zip(list_of_files, list_of_senses):
      # Open txt and read every line
      txt = open(path_in_wiktionary + file, "r", encoding = 'utf-8')
      lines = txt.readlines()
      txt.close()

      # Remove /n at the end of each line
      for index, line in enumerate(lines):
            lines[index] = line.strip()

      # print(lines[:10])

      # Creating a dataframe
      df = pd.DataFrame(columns=('first', 'second'))
      i = 0  
      first = "" 
      second = ""  
      for line in lines:
            # define what are the values in columns, e.g. second colum is extracted from line:
            second = re.sub(r"", "", line)
            # next line data
            df.loc[i] = [first, second]
            i =i+1

      # Replace empty cell with NaN
      df['second'].replace('', np.nan, inplace=True)

      # Drop NAs
      df.dropna(axis=0, inplace=True)

      # Reset index
      df.reset_index(drop=True, inplace=True)

      # Drop column?
      df.drop(['first'], axis=1, inplace=True)

      # Check if contains colon?
      df = df[df.second.str.contains(":", regex=True)]

      # Split columns along colons 
      df = pd.DataFrame(df.second.str.split(':', 1).tolist(), columns = ['language','term'])

      # Change empty to NaN
      # df.term[df['term']==""] = np.NaN
      df.term[df['term']==""] = "TOP"

      # Fill empty cells backward (i.e fill the parent language content with value from a variant if the former is empty) # NOT A GOOD SOLUTION, FIX MANUALLY SOMEHOW
      df['term'] =  df.term.str.extract('(.*)').fillna(method='bfill')

      # Add sense
      df['sense'] = sense

      # Concatenate frames
      df_key = pd.concat([df_key, df])

df = df_key #.copy()
print(df.shape[0])
df

229


,language,term,sense
0,Albanian,mëndafsh (sq) m,fabric
1,Arabic,حَرِير‎ m (ḥarīr),fabric
2,Egyptian Arabic,حرير‎ m (ḥarīr),fabric
3,Armenian,մետաքս (hy) (metakʿs),fabric
4,Aromanian,mãtase f,fabric
...,...,...,...
122,Walloon,soye (wa) f,fiber
123,Waray-Waray,sida,fiber
124,Welsh,sidan (cy) m,fiber
125,Westrobothnian,"selk n, siltj n",fiber


In [107]:
# # Sort by categories
# df['sense'] = pd.Categorical(df['sense'], list_of_senses) 
# df.sort_values("sense", inplace = True)

# # Sort by language
# df = df.sort_values('language')

# df.head(60)

In [108]:
# Drop duplicates
df = df.drop_duplicates(subset = ['language', 'term'], keep = 'first').reset_index(drop = True)

# Reset index
df.reset_index(inplace=True, drop=True)

# Filter duplicates among senses and get rid of B name if A is found. # Change subset to be more strict e.g. subset=['language', 'term']
if len(list_of_senses) > 1:
    df['duplicate'] = df[df.duplicated(subset=['language'], keep=False)]['sense'] == list_of_senses[1]
    df.drop(df[df['duplicate'] == True].index, inplace=True)
    df.drop(['duplicate'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)

print(df.shape[0])
df

146


,language,term,sense
0,Albanian,mëndafsh (sq) m,fabric
1,Arabic,حَرِير‎ m (ḥarīr),fabric
2,Egyptian Arabic,حرير‎ m (ḥarīr),fabric
3,Armenian,մետաքս (hy) (metakʿs),fabric
4,Aromanian,mãtase f,fabric
...,...,...,...
141,Volapük,sadin (vo),fiber
142,Walloon,soye (wa) f,fiber
143,Waray-Waray,sida,fiber
144,Welsh,sidan (cy) m,fiber


### Cleaning

In [109]:
# Cleaning
df['term'] = [re.sub('\xa0', " ", str(x)) for x in df['term']]

df['term'] = [re.sub(r' m ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(bcl\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(nds\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(scn\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(ast\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(Föhr-Amrum\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s?\(\w\w\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(please verify\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s+', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' ,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r'^\s', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s$', "", str(x)) for x in df['term']]

In [110]:
# Transliteration, formatting
df['transliteration'] = df.term.str.findall("(\((\w*\,?\.?\-?\:?\d?\s?ި?)+\))").fillna(method='ffill')

df['transliteration'] = [re.sub(r"[\(\)\[\]]", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r", ''", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"^'", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"'$", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"', '", ", ", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"ި", "", str(x)) for x in df['transliteration']]

# Other
df['transliteration'] = [re.sub(r", taraškievica", "", str(x)) for x in df['transliteration']]

# Clean term of transliteration
df['term'] = [re.sub(r"(\((\w*\,?\.?\-?\:?\d?\s?)+\))", "", str(x)) for x in df['term']] 
df['term'] = [re.sub(r" +,", ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!

In [111]:
# drop NA
df = df[df.term != "please add this translation if you can"]
print(df.shape[0])

# Create 'item' column where everything is in
df['item'] = df.loc[:, 'transliteration']
df['item'] = df['item'].replace('', pd.NA).fillna(df['term'])

# Create source
df['source'] = 'Wiktionary'
df['group'] = ''
df['skip'] = ''

# reorder
df = df[['skip', 'language', 'term', 'transliteration', 'item', 'group', 'sense', 'source']]
df.head(60)
df[110:114]

144


,skip,language,term,transliteration,item,group,sense,source
111,,Cornish,owrlin,,owrlin,,fiber,Wiktionary
112,,Corsican,seta,,seta,,fiber,Wiktionary
113,,Dhivehi,ފަށުި‎,faṣu,faṣu,,fiber,Wiktionary
114,,Esperanto,silko,,silko,,fiber,Wiktionary


In [112]:
# Change languages to Glottolog name
def glottologize():
    df['language'] = [re.sub(r"^Arabic$", "Standard Arabic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Armenian$", "Eastern Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Armenian (Eastern)$", "Eastern Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Armenian (Western)$", "Western Armenian", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Bikol Central$", "Coastal-Naga Bikol", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Bosnian$", "Bosnian Standard", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Croatian$", "Croatian Standard", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Gaelic$", "Scottish Gaelic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Greek$", "Modern Greek", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Greenlandic$", "Kalaallisut", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Hebrew$", "Modern Hebrew", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Kyrgyz$", "Kirghiz", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Luxembourgish$", "Luxemburgish", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Mandarin$", "Mandarin Chinese", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Manipuri (Meitei-Lon)$", "Manipuri", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Naga (Sumi)$", "Sumi Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Tangkhul)$", "North-Central Tangkhul Naga", str(x)) for x in df['language']] # A hypoglot
    df['language'] = [re.sub(r"^Naga (Rengma)$", "Northern Rengma Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Lotha)$", "Lotha Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Konyak)$", "Konyak Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Chakhesang-Chokri)$", "Chokri Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Ao)$", "Ao Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Angami)$", "Angami Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Norman$", "Anglo-Norman", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^North Frisian$", "Northern Frisian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^North Sami$", "North Saami", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Old Armenian$", "Classical-Middle Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old Church Slavonic$", "Church Slavic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old East Slavic$", "Old Russian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old Javanese$", "Kawi", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Ossetian$", "Modern Ossetic", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Punjabi$", "Eastern Panjabi", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^Serbian$", "Serbian Standard", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Serbo-Croatian$", "Serbian-Croatian-Bosnian", str(x)) for x in df['language']]

    df['language'] = [re.sub(r"^West Frisian$", "Western Frisian", str(x)) for x in df['language']]

    return df

glottologize()

,skip,language,term,transliteration,item,group,sense,source
0,,Albanian,mëndafsh,,mëndafsh,,fabric,Wiktionary
1,,Standard Arabic,حَرِير‎,ḥarīr,ḥarīr,,fabric,Wiktionary
2,,Egyptian Arabic,حرير‎,ḥarīr,ḥarīr,,fabric,Wiktionary
3,,Eastern Armenian,մետաքս,metakʿs,metakʿs,,fabric,Wiktionary
4,,Aromanian,mãtase,,mãtase,,fabric,Wiktionary
...,...,...,...,...,...,...,...,...
141,,Volapük,sadin,,sadin,,fiber,Wiktionary
142,,Walloon,soye,,soye,,fiber,Wiktionary
143,,Waray-Waray,sida,,sida,,fiber,Wiktionary
144,,Welsh,sidan,,sidan,,fiber,Wiktionary


In [113]:
# Write
df.to_excel(path_in_wiktionary + key + '_generated.xlsx', sheet_name='wiktionary', index=None, encoding="utf-8")

## Manual steps

Now the manual work: Use the generated file `key_generated.xlsx` to fix, amend, append, group, and organize the names, with the addition of other sources and create a `key.xlsx` master list.

Steps:
 1. Manual check transliterations, especially Dhivehi.
 2. Analyze and group words/names.
 3. Mark uncertain ones for skipping with 'yes'.

Recommended sources:
* Katzer (needs serious checking)
* WOLD
* Max Planck databases (CLIC3, etc.)
* others... 

After that, more preprocessing, cleaning, and merging with language data and coordinates.

In [ ]:
# Read and store content of an excel file 
read_file = pd.read_excel(path_in_wiktionary + key +".xlsx")

# Write the dataframe object into csv file
read_file.to_csv (path_in_wiktionary + key + ".csv", index = None, header=True)

# Load in dataset
df = pd.read_csv(path_in_wiktionary + key + ".csv", header =[0], delimiter=',', encoding="utf-8")

# Extract only desired columns
selectlist = ['skip', 'language', 'term', 'transliteration', 'item', 'group']
df = df[selectlist]

print("Before skipping: ", df.shape[0])
df=df[df['skip'] != "yes"]
print("After skipping: ", df.shape[0])

# # drop columns manually
# df.drop(columns=['skip', 'literal', 'explanation', 'IPA', 'source zotero', 'notes', 'type', 'katzer', 'katzer tr', 'checked', 'reference', 'link'], inplace=True)

# FIX BELOW HERE

In [114]:
# df_input = df.copy()

# # load in datasets
# languages=pd.read_csv(path_in_wiktionary + 'languages/languages.csv', header =[0], delimiter=',', encoding="utf-8", index_col=[0])

# # merge input and languages
# df = pd.merge(df_input, languages, on=['language'])
# print("Merged:", df.shape)

# #drop duplicates
# df.drop_duplicates(subset=['language', 'term'], keep='first', inplace=True, ignore_index=True)
# print("Dropping duplicates:", df.shape)

# multilingual = df
# multilingual

# #check missing ones
# temp = pd.merge(df_input, multilingual, how='outer', suffixes=('','_y'), indicator=True)
# missing = temp[temp['_merge']=='left_only'][df_input.columns]
# print("The following terms and languages have failed to load:")
# print(missing)

# # df = df.dropna() #OPERATIVE ONLY
# # df = df.fillna('x')

# #sort by categories, cinnamon ######## AUTOMATE ########
# df['group'] = pd.Categorical(df['group'], ["canela", "kinnamon", "korica", "qirfa", "darchin", "gui", "other"]) # add categorical order here
# df.sort_values("group", inplace = True) # sort according to the categories

# # #sort by categories, pepper ######## AUTOMATE ########
# # df['group'] = pd.Categorical(df['group'], ["pippali", "pigment", "marica", "hujiao", "other"]) # add categorical order here
# # df.sort_values("group", inplace = True) # sort according to the categories

# # create text for annotation label
# df['text'] = df['term'] + '<br>' + df['transliteration'].astype(str) + '<br>Language: ' + df['language'] + '<br>Family: ' + df['family']
# df['text'] = [re.sub(r"<br>nan<br>", "<br>", str(x)) for x in df['text']]

# df['term'] = [re.sub(r"\u200e", "", str(x)) for x in df['term']] #removes right to left mark
# df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!

# # reindex?

# # save
# df.to_csv(path_in_wiktionary + "multilingual/" + key +'.csv')
# df

### Multilingual extractions for Spices

In [ ]:
key = "cinnamon"

In [ ]:
# Open txt and separate every line
txt = open(path_in_wiktionary + key + '_spice.txt', "r", encoding = 'utf8')
lines = txt.readlines()
txt.close()

# Remove /n at the end of each line
for index, line in enumerate(lines):
      lines[index] = line.strip()

# Creating a dataframe
df = pd.DataFrame(columns=('first', 'second'))
i = 0  
first = "" 
second = ""  
for line in lines:
        #you have to kind of define what are the values in columns,for example second column includes:
        second = re.sub(r'', "", line)
        #this is how you create next line data
        df.loc[i] = [first, second]
        i =i+1


df['second'].replace('', np.nan, inplace=True)
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(['first'], axis=1, inplace=True)
df=df[df.second.str.contains(":", regex=True)]
df = pd.DataFrame(df.second.str.split(':', 1).tolist(), columns = ['language','term'])

df['sense'] = 'spice'

#fill empty cells backward (i.e fill the parent language content with value from a variant if the former is empty)
df.term[df['term']==""] = np.NaN
df['term'] =  df.term.str.extract('(.*)').fillna(method='bfill')

# df = df.sort_values('language')

spice = df
print(spice)

           language                                             term  sense
0         Afrikaans                                           kaneel  spice
1          Albanian                                   kanellë (sq) f  spice
2           Amharic                                     ቀረፋ (ḳäräfa)  spice
3            Arabic                               قِرْفَة‎ f (qirfa)  spice
4   Egyptian Arabic                                  قرفة‎ f (ʾerfa)  spice
..              ...                                              ...    ...
88            Uzbek                            dolchin (uz), koritsa  spice
89       Vietnamese                                         quế (vi)  spice
90          Volapük                                           kirfat  spice
91            Welsh                                        synamon m  spice
92          Yiddish   צימרינג‎ m (tsimring), צימערינג‎ m (tsimering)  spice

[93 rows x 3 columns]


In [ ]:
path = path_in_wiktionary + key + '_plant.txt'

if os.path.exists(path):

      #open txt and seperate every line
      df = open(path_in_wiktionary + key + '_plant.txt', "r", encoding = 'utf8')
      lines = df.readlines()
      df.close()

      # remove /n at the end of each line
      for index, line in enumerate(lines):
            lines[index] = line.strip()

      #creating a dataframe(consider u want to convert your data to 2 columns)
      df = pd.DataFrame(columns=('first', 'second'))
      i = 0 
      first = "" 
      second = ""  
      for line in lines:
            #you have to kind of define what are the values in columns,for example second column includes:
            second = re.sub(r'', "", line)
            #this is how you create next line data
            df.loc[i] = [first, second]
            i =i+1

      df['second'].replace('', np.nan, inplace=True)
      df.dropna(axis=0, inplace=True)
      df.reset_index(drop=True, inplace=True)
      df.drop(['first'], axis=1, inplace=True)
      df=df[df.second.str.contains(":", regex=True)]
      df = pd.DataFrame(df.second.str.split(':', 1).tolist(), columns = ['language','term'])

      df['sense'] = 'plant'

      #fill empty cells backward (i.e fill the parent language content with value from a variant)
      df.term[df['term']==""] = np.NaN
      df['term'] =  df.term.str.extract('(.*)').fillna(method='bfill')
      # df = df.sort_values('language')

      plant = df
      print(plant)

           language                                   term  sense
0         Afrikaans                             kaneelboom  plant
1            Arabic                     قِرْفَة‎ f (qirfa)  plant
2     Hijazi Arabic                          قرفة‎ (girfa)  plant
3   Moroccan Arabic                          قرفة‎ (qarfa)  plant
4           Aramaic   ܕܪܨܝܢܝ‎ (dārṣīnī), ܨܝܢܕܪܓ‎ (ṣīndreḡ)  plant
..              ...                                    ...    ...
61             Thai                       อบเชย (òp-chəəi)  plant
62          Tibetan                     ཤིང་ཚ (shing tsha)  plant
63          Turkish                           tarçın ağacı  plant
64       Vietnamese                                cây quế  plant
65          Volapük                   kirfatep, kirfatabim  plant

[66 rows x 3 columns]


In [ ]:
# concat
frames = [spice, plant]
df = pd.concat(frames)

# sort by categories
df['sense'] = pd.Categorical(df['sense'], ["spice", "plant"]) # add categorical order here
df.sort_values("sense", inplace = True) # sort according to the categories

# sort
df = df.sort_values('language')

# drop duplicates
df = df.drop_duplicates(subset = ['language', 'term'], keep = 'first').reset_index(drop = True)

# reset index
df.reset_index(inplace=True, drop=True)

# filter duplicates and get rid of plant name if spice name found
df['duplicate'] = df[df.duplicated(subset=['language'], keep=False)]['sense']=="plant" # change subset to be more strict e.g. subset=['language', 'term']
df.drop(df[df['duplicate'] == True].index, inplace=True)
df.drop(['duplicate'], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,language,term,sense
0,Afrikaans,kaneel,spice
1,Albanian,kanellë (sq) f,spice
2,Amharic,ቀረፋ (ḳäräfa),spice
3,Ancient,κιννάμωμον n (kinnámōmon),spice
4,Arabic,قِرْفَة‎ f (qirfa),spice
...,...,...,...
104,Uzbek,"dolchin (uz), koritsa",spice
105,Vietnamese,quế (vi),spice
106,Volapük,kirfat,spice
107,Welsh,synamon m,spice


In [ ]:
# cleaning
df['term'] = [re.sub(r' m ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(bcl\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(nds\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(scn\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(ast\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(Föhr-Amrum\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s?\(\w\w\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(please verify\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s+', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' ,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r'^\s', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s$', "", str(x)) for x in df['term']]

# transliteration, formatting
df['transliteration'] = df.term.str.findall("(\((\w*\,?\.?\-?\:?\d?\s?)+\))").fillna(method='ffill')
df['transliteration'] = [re.sub(r"[\(\)\[\]]", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r", ''", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"^'", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"'$", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"', '", ", ", str(x)) for x in df['transliteration']]

# clean term of transliteration
df['term'] = [re.sub(r"(\((\w*\,?\.?\-?\:?\d?\s?)+\))", "", str(x)) for x in df['term']] 
df['term'] = [re.sub(r" +,", ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!


# other
df['transliteration'] = [re.sub(r", taraškievica", "", str(x)) for x in df['transliteration']]

# drop NA
df = df[df.term != "please add this translation if you can"]

# create 'item' column where everythin is in
df['item'] = df['transliteration']
df['item'] = df['item'].replace('', pd.NA).fillna(df['term'])
df['source zotero'] = 'Wiktionary'

# reorder
df = df[['language', 'term', 'transliteration', 'item', 'sense']]

# Change languages to glottolog name
df['language'] = [re.sub(r"^Arabic$", "Standard Arabic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Mandarin$", "Mandarin Chinese", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Armenian$", "Classical-Middle Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Croatian$", "Croatian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Serbian$", "Serbian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Bosnian$", "Bosnian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Serbo-Croatian$", "Serbian-Croatian-Bosnian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Gaelic$", "Scottish Gaelic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Greek$", "Modern Greek", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Punjabi$", "Eastern Panjabi", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Norman$", "Anglo-Norman", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Bikol Central$", "Coastal-Naga Bikol", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian (Eastern)$", "Eastern Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian (Western)$", "Western Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Manipuri (Meitei-Lon)$", "Manipuri", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Sumi)$", "Sumi Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Tangkhul)$", "North-Central Tangkhul Naga", str(x)) for x in df['language']] # A hypoglot
df['language'] = [re.sub(r"^Naga (Rengma)$", "Northern Rengma Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Lotha)$", "Lotha Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Konyak)$", "Konyak Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Chakhesang-Chokri)$", "Chokri Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Ao)$", "Ao Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Angami)$", "Angami Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Kyrgyz$", "Kirghiz", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Hebrew$", "Modern Hebrew", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Northern Sami$", "North Sami", str(x)) for x in df['language']]



#write
df.to_excel(path_in_wiktionary + key + '_generated.xlsx', sheet_name='wiktionary', index=None, encoding="utf-8")
df

,language,term,transliteration,item,sense
0,Afrikaans,kaneel,,kaneel,spice
1,Albanian,kanellë,,kanellë,spice
2,Amharic,ቀረፋ,ḳäräfa,ḳäräfa,spice
3,Ancient,κιννάμωμον,kinnámōmon,kinnámōmon,spice
4,Standard Arabic,قِرْفَة‎,qirfa,qirfa,spice
...,...,...,...,...,...
104,Uzbek,"dolchin, koritsa",,"dolchin, koritsa",spice
105,Vietnamese,quế,,quế,spice
106,Volapük,kirfat,,kirfat,spice
107,Welsh,synamon,,synamon,spice


Now the manual work: Use `spice_out.xlsx` to fix, amend, append, group, and organize the names, with the addition of other sources and create a `spice.xlsx` master list.

Recommended sources:
* Katzer (needs serious checking)
* WOLD
* Max Planck databases (CLIC3, etc.)
* others... 

After that, more preprocessing, cleaning, and merging with language data and coordinates.

In [ ]:
# Read and store content of an excel file 
read_file = pd.read_excel(path_in_wiktionary + key +".xlsx")

# Write the dataframe object into csv file
read_file.to_csv (path_in_wiktionary + key + ".csv", index = None, header=True)

# Load in dataset
df=pd.read_csv(path_in_wiktionary + key + ".csv", header =[0], delimiter=',', encoding="utf-8")

# extract only desired columns
selectlist = ['skip', 'language', 'term', 'transliteration', 'item', 'group']
df = df[selectlist]

print("Before skipping: ", df.shape)
df=df[df['skip'] != "yes"]
print("After skipping: ", df.shape)

# # drop columns manually
# df.drop(columns=['skip', 'literal', 'explanation', 'IPA', 'source zotero', 'notes', 'type', 'katzer', 'katzer tr', 'checked', 'reference', 'link'], inplace=True)

# Change languages to glottolog name
df['language'] = [re.sub(r"^Arabic$", "Standard Arabic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Mandarin$", "Mandarin Chinese", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Armenian$", "Classical-Middle Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Croatian$", "Croatian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Serbian$", "Serbian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Bosnian$", "Bosnian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Serbo-Croatian$", "Serbian-Croatian-Bosnian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Gaelic$", "Scottish Gaelic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Greek$", "Modern Greek", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Punjabi$", "Eastern Panjabi", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Norman$", "Anglo-Norman", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Bikol Central$", "Coastal-Naga Bikol", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian (Eastern)$", "Eastern Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian (Western)$", "Western Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Manipuri (Meitei-Lon)$", "Manipuri", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Sumi)$", "Sumi Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Tangkhul)$", "North-Central Tangkhul Naga", str(x)) for x in df['language']] # A hypoglot
df['language'] = [re.sub(r"^Naga (Rengma)$", "Northern Rengma Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Lotha)$", "Lotha Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Konyak)$", "Konyak Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Chakhesang-Chokri)$", "Chokri Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Ao)$", "Ao Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Angami)$", "Angami Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Kyrgyz$", "Kirghiz", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Hebrew$", "Modern Hebrew", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^North Sami$", "North Saami", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Greenlandic$", "Kalaallisut", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^West Frisian$", "Western Frisian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Javanese$", "Kawi", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Church Slavonic$", "Church Slavic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^North Frisian$", "Northern Frisian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Luxembourgish$", "Luxemburgish", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian$", "Eastern Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old East Slavic$", "Old Russian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Ossetian$", "Modern Ossetic", str(x)) for x in df['language']]

df_input = df

# load in datasets
languages=pd.read_csv(path_in_wiktionary + 'languages/languages.csv', header =[0], delimiter=',', encoding="utf-8", index_col=[0])

# merge input and languages
df = pd.merge(df_input, languages, on=['language'])
print("Merged:", df.shape)

#drop duplicates
df.drop_duplicates(subset=['language', 'term'], keep='first', inplace=True, ignore_index=True)
print("Dropping duplicates:", df.shape)

multilingual = df
multilingual

#check missing ones
temp = pd.merge(df_input, multilingual, how='outer', suffixes=('','_y'), indicator=True)
missing = temp[temp['_merge']=='left_only'][df_input.columns]
print("The following terms and languages have failed to load:")
print(missing)

# df = df.dropna() #OPERATIVE ONLY
# df = df.fillna('x')

#sort by categories, cinnamon ######## AUTOMATE ########
df['group'] = pd.Categorical(df['group'], ["canela", "kinnamon", "korica", "qirfa", "darchin", "gui", "other"]) # add categorical order here
df.sort_values("group", inplace = True) # sort according to the categories

# #sort by categories, pepper ######## AUTOMATE ########
# df['group'] = pd.Categorical(df['group'], ["pippali", "pigment", "marica", "hujiao", "other"]) # add categorical order here
# df.sort_values("group", inplace = True) # sort according to the categories

# create text for annotation label
df['text'] = df['term'] + '<br>' + df['transliteration'].astype(str) + '<br>Language: ' + df['language'] + '<br>Family: ' + df['family']
df['text'] = [re.sub(r"<br>nan<br>", "<br>", str(x)) for x in df['text']]

df['term'] = [re.sub(r"\u200e", "", str(x)) for x in df['term']] #removes right to left mark
df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!

# reindex?

# save
df.to_csv(path_in_wiktionary + "multilingual/" + key +'.csv')
df

Before skipping:  (162, 6)
After skipping:  (148, 6)
Merged: (155, 16)
Dropping duplicates: (148, 16)
The following terms and languages have failed to load:
Empty DataFrame
Columns: [skip, language, term, transliteration, item, group]
Index: []


,skip,language,term,transliteration,item,group,iso,glcode,level,branch,family,macroarea,country,lat,lon,from,text
0,NaN,Afrikaans,kaneel,NaN,kaneel,canela,afr,afri1274,language,Germanic,Indo-European,Africa,ZA,-31.00000,22.00000,wals1,kaneel<br>Language: Afrikaans<br>Family: Indo-...
36,NaN,Estonian,kaneel,NaN,kaneel,canela,est,esto1258,language,Finnic,Uralic,Eurasia,EE,59.00000,26.00000,wals1,kaneel<br>Language: Estonian<br>Family: Uralic
37,NaN,Faroese,kanel,NaN,kanel,canela,fao,faro1244,language,Germanic,Indo-European,Eurasia,DK,62.00000,-7.00000,wals1,kanel<br>Language: Faroese<br>Family: Indo-Eur...
38,NaN,Finnish,kaneli,NaN,kaneli,canela,fin,finn1318,language,Finnic,Uralic,Eurasia,FI,62.00000,25.00000,wals1,kaneli<br>Language: Finnish<br>Family: Uralic
39,NaN,French,cannelle,NaN,cannelle,canela,fra,stan1290,language,Romance,Indo-European,Eurasia,CH FR,48.00000,2.00000,wals1,cannelle<br>Language: French<br>Family: Indo-E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,NaN,Sumi Naga,"losani, akusa",NaN,"losani, akusa",other,nsm,sumi1235,language,NaN,Sino-Tibetan,Eurasia,IN,25.99960,94.42350,glot,"losani, akusa<br>Language: Sumi Naga<br>Family..."
12,NaN,Brunei Malay,kayu manis,NaN,kayu manis,other,NaN,brun1243,dialect,NaN,Austronesian,Papunesia,NaN,4.53000,114.72000,glot,kayu manis<br>Language: Brunei Malay<br>Family...
146,NaN,Yoruba,eso igi gbigbẹ oloorun,NaN,eso igi gbigbẹ oloorun,other,yor,yoru1245,language,NaN,Atlantic-Congo,Africa,BJ NG,7.15345,3.67225,glot,eso igi gbigbẹ oloorun<br>Language: Yoruba<br>...
71,NaN,Lao,ອົບເຊຍ,obsey,obsey,other,lao,laoo1244,language,Kam-Tai,Tai-Kadai,Eurasia,LA TH,18.00000,103.00000,wals1,ອົບເຊຍ<br>obsey<br>Language: Lao<br>Family: Ta...


# Analysis